# Тестовое задание УЖЦК

В данном тестовом задании предлагается выполнить несколько прикладных задач, связанных с псевдоданными УЖЦК.

### Описание данных

В приложении к заданию есть файл database.db. Он представляет собой файл БД на фреймворке sqlite3. Синтаксис SQL, используемый в данной БД почти не отличается от ORACLE DB (более подробно можно почитать тут: https://unetway.com/tutorial/sqlite-syntax).
Запросы к базе можно сохранять сразу в pandas.Dataframe, используя следующий код:

In [21]:
import pandas as pd
import sqlite3 as sql

query_pl = 'SELECT * FROM pl'
con = sql.connect('database.db')
df_pl = pd.read_sql_query(sql = query_pl, con=con)
con.close()

df_pl




,client_id,segment,period,product,pl
0,ZDosW1zUAi,Массовый,p_1,Кредит,9.018067
1,wVRHQ9hzrw,Массовый,p_1,Кредит,18.889585
2,6DZQIaOyL3,Массовый,p_1,Кредит,0.369257
3,jDwO0sMhL2,Массовый,p_1,Кредит,124.315319
4,FXIJUkgjLy,Массовый,p_1,Кредит,41.287831
...,...,...,...,...,...
92604,G8E6dyKT3k,Массовый,p_2,Банковская карта,17.362040
92605,10OPeJCF1l,Массовый,p_2,Банковская карта,1.861434
92606,BMN762bZh5,Верхний,p_2,Банковская карта,78.430698
92607,FcB9pJHGVX,Стандартный,p_2,Банковская карта,16.107313


   В базе данных хранится 2 таблицы: pl и campaign. Ключами для связи этих таблиц является поле client_id. В таблице pl находится информация о ЧОД (чистый операционный доход) от клиента с продуктом в разрезе периодов. В таблице campaign хранятся данные о прошедшей  между отчетными периодами "p_1" и "p_2" маркетинговой кампании.<br>
Атрибутный состав таблицы pl:
   - client_id - id клиента
   - segment - сегмент клиента внутри СМБ(Средний и малый бизнес)
   - period - период расчета ЧОД банка от продукта, используемого клиентом
   - product - продукт 
   - pl - ЧОД в тысячах рублей
   
Пример: строка 0 на срезе выше - в периоде "p_1" за продукт "Кредит" с клиента "yxwtZv530y" массового сегмента банк заработал 0.374924 тыс. рублей.
   
Если в каком-либо периоде информация о клиенте/продукте у клиента отсутствует, значит в этот период клиента в банке не было/клиент продуктом не пользовался.

Атрибутный состав таблицы campaign:
- client_id - id клиента
- product - продукт кампании
- response - статус отклика на предложение
- manager_id - id менеджера, на которого назначен клиент 

### Задание 

1) Посчитать средний ЧОД по продуктам и количество клиентов с продуктом для каждого сегмента в разрезе периодов.<br>
2) Посчитать средний ЧОД и количество клиентов в каждом сегменте в разрезе периодов.<br>
3) Оценить отток клиентов с продуктов в периоде p_2.<br>
4) Посчитать сумму ЧОД, привлеченную каждым менеджером в ходе маркетинговой кампании<br>
5) Оценить эффективность маркетинговой кампании.

In [32]:
# Посчитать средний ЧОД по продуктам и количество клиентов с продуктом для каждого сегмента в разрезе периодов

con = sql.connect('database.db')
query = '''
SELECT 
    period,
    segment,
    product,
    COUNT(DISTINCT client_id) AS client_count,
    AVG(pl) AS average_pl
FROM 
    pl
GROUP BY 
    period, segment, product
ORDER BY 
    period, segment, product;
'''
df_results = pd.read_sql_query(query, con)

# Выводим результаты
print(df_results)


   period      segment           product  client_count  average_pl
0     p_1      Верхний  Банковская карта            49   60.571545
1     p_1      Верхний           Депозит           254  112.474520
2     p_1      Верхний            Кредит           360  236.938710
3     p_1      Верхний               РКО          1149   22.847632
4     p_1      Верхний         Эквайринг           183   80.517184
5     p_1     Массовый  Банковская карта           895   13.310203
6     p_1     Массовый           Депозит          3537   28.747545
7     p_1     Массовый            Кредит          5247   59.219407
8     p_1     Массовый               РКО         16728    5.795114
9     p_1     Массовый         Эквайринг          2625   19.627608
10    p_1  Стандартный  Банковская карта           338   26.421616
11    p_1  Стандартный           Депозит          1292   60.994262
12    p_1  Стандартный            Кредит          1871  120.101219
13    p_1  Стандартный               РКО          5907   11.41

In [39]:
# 2) Посчитать средний ЧОД и количество клиентов в каждом сегменте в разрезе периодов.<br>

conn = sql.connect('database.db')

querry = '''

SELECT
    segment,
    period,
    COUNT(DISTINCT client_id) as client_count,
    AVG(pl) as average_pl    
FROM
    pl
GROUP BY
    period, segment 
ORDER BY
    period, segment
'''

df_results = pd.read_sql_query(querry, conn)

# Выводим результаты
print(df_results)



       segment period  client_count  average_pl
0      Верхний    p_1          1195   79.108293
1     Массовый    p_1         17207   19.729266
2  Стандартный    p_1          6061   40.282587
3      Верхний    p_2          1206   89.650947
4     Массовый    p_2         17330   23.273464
5  Стандартный    p_2          6110   46.503397


In [70]:
# 3. Оценить отток клиентов с продуктов в периоде p_2

conn = sql.connect('database.db')

querry_p1 = '''
SELECT 
    product,
    client_id
FROM 
    pl
WHERE
    period = 'p_1'
'''
df_p1 = pd.read_sql_query(querry_p1, conn)


querry_p2 = '''
SELECT 
    product,
    client_id
FROM 
    pl
WHERE
    period = 'p_2'
'''
df_p2 = pd.read_sql_query(querry_p2, conn)
# print(df_p1)
# print(df_p2)


# Определим отток клиентов по продуктам
# Найдем уникальных клиентов для каждого продукта в p_1 и p_2
clients_p1 = df_p1.groupby('product')['client_id'].unique().reset_index(name='clients_p1')
clients_p2 = df_p2.groupby('product')['client_id'].unique().reset_index(name='clients_p2')
# print(clients_p1)

# Объединим данные по продуктам
clients_data = pd.merge(clients_p1, clients_p2, on='product', how='left')

# Рассчитаем количество клиентов в p_1 и p_2 и определим отток
clients_data['clients_count_p1'] = clients_data['clients_p1'].apply(len)
clients_data['clients_count_p2'] = clients_data['clients_p2'].apply(len)

# Рассчитаем отток
clients_data['clients_out'] = clients_data['clients_count_p1'] - clients_data['clients_count_p2']
clients_data['clients_out_rate'] = (clients_data['clients_out'] / clients_data['clients_count_p1']) * 100


print(clients_data)


            product                                         clients_p1  \
0  Банковская карта  [jDwO0sMhL2, u1OidVN4yp, xAJG2uVwXU, sRvtRR8lN...   
1           Депозит  [XHgfHt3boG, sujQEPdgvp, eZ1zY4mbSa, oRrfFfoid...   
2            Кредит  [ZDosW1zUAi, wVRHQ9hzrw, 6DZQIaOyL3, jDwO0sMhL...   
3               РКО  [vB041WM4xw, SAN5qFF3Qg, ZDosW1zUAi, wVRHQ9hzr...   
4         Эквайринг  [eZ1zY4mbSa, AmbDMtMnUr, vrW5Qldpj4, GjqWSnMK1...   

                                          clients_p2  clients_count_p1  \
0  [wVRHQ9hzrw, uV4WvJ5YkD, FXIJUkgjLy, NvY3CPB2M...              1282   
1  [SAN5qFF3Qg, wVRHQ9hzrw, cfbHjioHGx, 6DZQIaOyL...              5083   
2  [XHgfHt3boG, sujQEPdgvp, jj7zjaRO9L, FXIJUkgjL...              7478   
3  [vB041WM4xw, SAN5qFF3Qg, ZDosW1zUAi, wVRHQ9hzr...             23784   
4  [SAN5qFF3Qg, uV4WvJ5YkD, XHgfHt3boG, jj7zjaRO9...              3770   

   clients_count_p2  clients_out  clients_out_rate  
0              3720        -2438       -190.171607  
1   

Видим, что отток клиентов в период p2 произошел только на продукте РКО. Остальные продукты показали значительный рост - от 21% до 190%!

In [94]:
# 4) Посчитать сумму ЧОД, привлеченную каждым менеджером в ходе маркетинговой кампании

conn = sql.connect('database.db')

query_mng = '''
SELECT 
    campaign.manager_id, campaign.product, campaign.client_id, AVG(pl) as average_pl
FROM 
    campaign
    JOIN pl ON campaign.client_id = pl.client_id
WHERE
    campaign.response = 'Согласие'
GROUP BY 
    manager_id
ORDER BY 
    average_pl DESC;
'''

df_res = pd.read_sql_query(query_mng, conn)

print(df_res)

       manager_id product   client_id  average_pl
0   CM_2755B4KTu4  Кредит  1AwiAwSFYb   38.391365
1   CM_PzEEav4ic0  Кредит  VWTKRllPHK   37.502652
2   CM_pMFgOhtUKD  Кредит  Vc00RX3ls2   37.256653
3   CM_70HjN0EVqf  Кредит  9t3ipsWw7E   37.174437
4   CM_wjb2ArrpAw  Кредит  rnOUYpL9k2   36.893391
5   CM_R7BdTh0Svt  Кредит  K25LPDVjHH   36.661391
6   CM_c28lGRRaEI  Кредит  3RUZMZleIv   36.233672
7   CM_nJ7j5pAcMB  Кредит  fx0acVADvj   35.755782
8   CM_6ofody41cY  Кредит  IEryxBXw38   35.452655
9   CM_gHP4ykbzTj  Кредит  u0CR0M2z38   34.558025
10  CM_zShYeVNFi0  Кредит  wsBibHoVUL   34.536252
11  CM_EiBeeRMhdA  Кредит  owOyJEIst9   34.205011
12  CM_eE0WMK1vVW  Кредит  0bojUFuq8e   33.828082
13  CM_onY6ha8qHv  Кредит  xyqS1Tl1j8   33.713924
14  CM_doxY9cvELI  Кредит  Efp9TiwFiB   33.424302
15  CM_9vMKq9aqw9  Кредит  nw2LOK9EBo   33.349508
16  CM_lKxhD73Z6d  Кредит  8iRfG4cr5M   33.126411
17  CM_RCwv1r0OHO  Кредит  x1rEdC5fys   32.370575
18  CM_dcTdLjqH2x  Кредит  gipmx9KwLD   32.138577


5) Оценить эффективность маркетинговой кампании.

Судя по притоку клиентов во втором периоде (p2), можно отметить успешность маркетинговой кампании. По итогам работы, можно выделить, например, топ-5 менеджеров, заключивших больше продаж.

